In [1]:
from fastai.text import *
from sqlalchemy import create_engine

In [2]:
with open('psql_engine.txt') as f:
    psql = create_engine(f.read())
    
with open('lm_gpu/data_ru/tmp/itos.pkl', 'rb') as fru, open('lm_gpu/data_uk/tmp/itos.pkl', 'rb') as fuk:
    itos_ru = pickle.load(fru)
    itos_uk = pickle.load(fuk)

In [3]:
to_check = '688276, 1731042, 2463925, 2493020, 938827, 1809868, 2889683, 290087, 1598581'

In [4]:
check_ids = pd.read_sql(f'''
    select html_id, tokenized, headline_tok, word_ids, lang
    from htmls
    where html_id in ({to_check});
    ''', psql)

In [5]:
%%capture
bptt,em_sz,nh,nl,c = 70,400,1150,3,2
vs = len(itos_ru)
# bs = 2

sigm = lambda x: 1 / (1 + np.exp(-x))
sigmv = np.vectorize(sigm)

m_pred = get_rnn_classifer(bptt, 25*70, 2, vs,
                           emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
                           drops=[0,0],
                           layers=[em_sz*3, 50, c])

load_model(m_pred, 'classifiers/arg_2_mult_best54.h5')

m_pred.cuda()
m_pred.eval()

In [6]:
ds_ru = TextDataset(check_ids.loc[check_ids.lang == 'ru'].word_ids.values, check_ids.loc[check_ids.lang == 'ru'].html_id.values)
dl_ru = DataLoader(ds_ru, 24, transpose=True, num_workers=3, pad_idx=1)
# pred = predict(m_pred, dl)
# res = sigmv(pred)

In [27]:
result = m_pred(V(next(iter(dl_ru))[0]))

preds_ru = sigmv(result[0].data.cpu().numpy())
ru_states = []
for i in range(len(ds_ru)):
    ru_states.append(result[-1][-1][:, i, :].data.cpu().numpy())
    
del result

In [36]:
BOS = 'xbos'
FLD = 'xbod'

def split_tok(text):
    return' \n '.join([p.replace('\n', f' {BOS} ')
                       for p in text.strip().split('\n\n')]
                     ).split(' ')

In [40]:
splitted_ru = (f'\n{BOS} {FLD} 1 ' + check_ids.loc[check_ids.lang == 'ru'].tokenized
              ).apply(split_tok).values.tolist()

In [92]:
states_of_news = {}
for splited, states, preds, html_id in zip(splitted_ru, ru_states, preds_ru, ds_ru.y):
    states_per_word = np.hstack([np.array(splited)[:, None], states[-len(splited):]]).tolist()
    states_of_news[str(html_id)] = {
        'states_per_word': states_per_word,
        'pred_emo': preds[0],
        'pred_arg': preds[1],
    }
    

In [93]:
with open('ru_states_per_word.json', 'w') as f:
    json.dump(states_of_news, f, ensure_ascii=False, allow_nan=False)